In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
%%time
coor_df = pd.read_csv('./prepped_data/coor_data_ver_2.csv')
print(coor_df.shape)
coor_df.head()

(567335, 3)
Wall time: 594 ms


,도로명주소,X좌표,Y좌표
0,서울특별시 종로구 자하문로 94-0,953241.683263,1.954023e+06
1,서울특별시 종로구 자하문로 96-0,953243.258151,1.954034e+06
2,서울특별시 종로구 자하문로 98-0,953240.877646,1.954055e+06
3,서울특별시 종로구 자하문로 99-3,953195.459364,1.954079e+06
4,서울특별시 종로구 자하문로 99-4,953193.346779,1.954097e+06


In [3]:
addr_df = pd.read_csv('./지번주소도로명주소/주소/지번_도로명_주소.csv')
print(addr_df.shape)
addr_df.head()

(803177, 22)


,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,산여부,지번본번,지번부번,대표여부,도로명코드,읍면동일련번호,지하여부,건물본번,건물부번,기초구역번호,상세주소부여 여부,도로명,읍면동명,읍면동구분,읍면동코드,사용여부
0,1111010100100010000030843,1,1111010100,서울특별시,종로구,청운동,0,1,0,1,111104100289,1,0,16,14,3046,1,자하문로36길,청운동,1,101.0,0
1,1111010100100010000030843,1,1111010100,서울특별시,종로구,청운동,0,1,0,1,111104100289,1,0,16,14,3046,0,자하문로36길,청운동,1,101.0,0
2,1111010100100030000030936,1,1111010100,서울특별시,종로구,청운동,0,3,0,1,111104100289,1,0,16,11,3046,1,자하문로36길,청운동,1,101.0,0
3,1111010100100030150030938,1,1111010100,서울특별시,종로구,청운동,0,3,150,1,111104100289,1,0,16,5,3046,1,자하문로36길,청운동,1,101.0,0
4,1111010100100040001000001,1,1111010100,서울특별시,종로구,청운동,0,4,1,1,111104100289,1,0,28,0,3046,1,자하문로36길,청운동,1,101.0,0


In [4]:
addr_df['지번주소'] = addr_df['시도명'] + ' ' + addr_df['시군구명'] + ' ' + addr_df['법정읍면동명'] + ' ' +\
addr_df['산여부'].apply(lambda x: '산' if x == 1 else '') + addr_df['지번본번'].apply(str) + '-' + addr_df['지번부번'].apply(str)

In [6]:
addr_df.drop(columns=['관리번호', '일련번호', '법정동코드', '대표여부', '도로명코드', '읍면동일련번호', '지하여부',
                      '기초구역번호', '상세주소부여 여부', '읍면동코드', '읍면동구분', '사용여부'
                     ], inplace=True)

In [7]:
addr_df.head()

,시도명,시군구명,법정읍면동명,산여부,지번본번,지번부번,건물본번,건물부번,도로명,읍면동명,지번주소
0,서울특별시,종로구,청운동,0,1,0,16,14,자하문로36길,청운동,서울특별시 종로구 청운동 1-0
1,서울특별시,종로구,청운동,0,1,0,16,14,자하문로36길,청운동,서울특별시 종로구 청운동 1-0
2,서울특별시,종로구,청운동,0,3,0,16,11,자하문로36길,청운동,서울특별시 종로구 청운동 3-0
3,서울특별시,종로구,청운동,0,3,150,16,5,자하문로36길,청운동,서울특별시 종로구 청운동 3-150
4,서울특별시,종로구,청운동,0,4,1,28,0,자하문로36길,청운동,서울특별시 종로구 청운동 4-1


In [8]:
addr_df['도로명주소'] = addr_df['시도명'] + ' ' + addr_df['시군구명'] + ' ' + addr_df['도로명'] + ' ' +\
addr_df['건물본번'].apply(str) + '-' + addr_df['건물부번'].apply(str)

In [9]:
addr_df.head()

,시도명,시군구명,법정읍면동명,산여부,지번본번,지번부번,건물본번,건물부번,도로명,읍면동명,지번주소,도로명주소
0,서울특별시,종로구,청운동,0,1,0,16,14,자하문로36길,청운동,서울특별시 종로구 청운동 1-0,서울특별시 종로구 자하문로36길 16-14
1,서울특별시,종로구,청운동,0,1,0,16,14,자하문로36길,청운동,서울특별시 종로구 청운동 1-0,서울특별시 종로구 자하문로36길 16-14
2,서울특별시,종로구,청운동,0,3,0,16,11,자하문로36길,청운동,서울특별시 종로구 청운동 3-0,서울특별시 종로구 자하문로36길 16-11
3,서울특별시,종로구,청운동,0,3,150,16,5,자하문로36길,청운동,서울특별시 종로구 청운동 3-150,서울특별시 종로구 자하문로36길 16-5
4,서울특별시,종로구,청운동,0,4,1,28,0,자하문로36길,청운동,서울특별시 종로구 청운동 4-1,서울특별시 종로구 자하문로36길 28-0


In [10]:
addr_df.loc[0, '도로명주소']

'서울특별시 종로구 자하문로36길 16-14'

In [11]:
selected_df = addr_df[['지번주소', '도로명주소']].drop_duplicates(subset=['지번주소']).reset_index(drop=True)
print(selected_df.shape)
selected_df.head()

(650680, 2)


,지번주소,도로명주소
0,서울특별시 종로구 청운동 1-0,서울특별시 종로구 자하문로36길 16-14
1,서울특별시 종로구 청운동 3-0,서울특별시 종로구 자하문로36길 16-11
2,서울특별시 종로구 청운동 3-150,서울특별시 종로구 자하문로36길 16-5
3,서울특별시 종로구 청운동 4-1,서울특별시 종로구 자하문로36길 28-0
4,서울특별시 종로구 청운동 3-94,서울특별시 종로구 자하문로36길 28-0


In [12]:
merge_df = selected_df.merge(coor_df, on=['도로명주소'])
print(merge_df.shape)
merge_df.head()

(626610, 4)


,지번주소,도로명주소,X좌표,Y좌표
0,서울특별시 종로구 청운동 1-0,서울특별시 종로구 자하문로36길 16-14,953042.185946,1.954799e+06
1,서울특별시 종로구 청운동 3-0,서울특별시 종로구 자하문로36길 16-11,953033.999264,1.954773e+06
2,서울특별시 종로구 청운동 3-150,서울특별시 종로구 자하문로36길 16-5,953028.717755,1.954717e+06
3,서울특별시 종로구 청운동 4-1,서울특별시 종로구 자하문로36길 28-0,952962.680291,1.954697e+06
4,서울특별시 종로구 청운동 3-94,서울특별시 종로구 자하문로36길 28-0,952962.680291,1.954697e+06


In [13]:
merge_df.to_csv('./prepped_data/addr_coor_data_ver_1.csv', index=False)